In [1]:
data_samples <- read.csv("data_melatonion.csv", check.names = TRUE)
dim(data_samples)
clock_stool <- read.csv("Clock_stool.csv", check.names = TRUE)


[1] 712   7

In [2]:

Study_variable <- 'Melatonin_pg_g_Stool'  
as.symbol(Study_variable)
Ages <- c('all' , '3 months', '6 months', '12 months')

Age <- Ages[1]     #'all' , '3 months', '6 months';, '12 months'


data_act <- read.csv("CFI_All.csv", check.names = TRUE)
#head(data_act)
dim(data_act)


Melatonin_pg_g_Stool

[1] 486   2

In [3]:
act_vars <- c('CFI')
act_var <- act_vars[1]


In [4]:
# Load necessary library
library(dplyr)

# Define the preprocessing function
convert_time_to_numeric <- function(df, act_var) {
  
  # Check each value in the column: if it's not numeric and contains ":", convert it
  df <- df %>%
    mutate(!!act_var := ifelse(grepl(":", as.character(!!sym(act_var))),
                               sapply(strsplit(as.character(!!sym(act_var)), ":"), 
                                      function(x) as.numeric(x[1]) + as.numeric(x[2]) / 60),
                               as.numeric(!!sym(act_var))))  # Keep numeric values as they are
  
  cat(paste(act_var, "has been checked and converted where needed.\n"))
  
  return(df)
}

# Example usage:
data_act <- convert_time_to_numeric(data_act, act_var)
clock_stool <- convert_time_to_numeric(clock_stool, 'Clock_stool')



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




CFI has been checked and converted where needed.


Warning message:
"There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Clock_stool = ifelse(...)`.
Caused by warning in `FUN()`:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."


Clock_stool has been checked and converted where needed.


In [5]:
data_act <- merge(data_act,clock_stool, by = "Sample_Code_16s")
data_act

Sample_Code_16s CFI       Clock_stool
1   F300-06         0.5783333  7.250000  
2   F300-06         0.5783333  6.833333  
3   F302-03         0.6633333 12.000000  
4   F302-03         0.6633333  7.750000  
5   F302-06         0.7083333  8.000000  
6   F302-06         0.7083333  9.833333  
7   F303-03         0.6916667 19.666667  
8   F303-03         0.6916667 15.750000  
9   F304-03         0.5950000 11.250000  
10  F304-03         0.5950000 10.000000  
11  F305-03         0.6583333  6.500000  
12  F305-03         0.6583333  8.500000  
13  F306-06         0.6833333  9.000000  
14  F306-06         0.6833333 18.000000  
15  F307-03         0.6816667 19.000000  
16  F307-03         0.6816667 18.116667  
17  F308-03         0.5750000 13.583333  
18  F308-03         0.5750000 11.500000  
19  F308-06         0.6300000 18.000000  
20  F308-06         0.6300000  9.750000  
21  F309-03         0.5433333 20.666667  
22  F309-03         0.5433333 12.000000  
23  F309-06         0.6016667 18.750000  
24  F309-06         0.6016667  8.333333  
25  F310-03         0.6133333 15.750000  
26  F310-03         0.6133333  7.333333  
27  F310-06         0.6733333 16.000000  
28  F310-06         0.6733333 11.233333  
29  I001-03         0.7683333 12.416667  
30  I001-03         0.7683333  8.166667  
⋮   ⋮               ⋮         ⋮          
475 S146-12         0.7150000 15.000000  
476 S147-03         0.6950000  8.500000  
477 S148-03         0.6750000 11.916667  
478 S148-06         0.6783333 20.583333  
479 S148-12         0.7116667 17.500000  
480 S149-03         0.5266667 17.666667  
481 S149-06         0.6116667  9.166667  
482 S149-12         0.8300000  8.200000  
483 S150-03         0.6450000 15.500000  
484 S150-06         0.7000000  6.916667  
485 S150-12         0.7516667  8.750000  
486 S152-03         0.5916667 18.000000  
487 S152-12         0.7600000  8.500000  
488 S153-03         0.6483333 10.000000  
489 S153-06         0.7250000 11.500000  
490 S153-12         0.7666667 17.000000  
491 S154-03         0.6183333  7.250000  
492 S154-06         0.6883333  7.916667  
493 S154-12         0.6416667  8.166667  
494 S156-03         0.5600000  3.383333  
495 S156-06         0.5450000  8.266667  
496 S156-12         0.7616667  8.450000  
497 S157-03         0.5650000  7.833333  
498 S157-06         0.5283333 11.500000  
499 S157-12         0.7116667  7.000000  
500 S158-03         0.5583333 15.500000  
501 S158-06         0.5983333  3.250000  
502 S158-12         0.7333333 12.500000  
503 S159-03         0.6166667 16.500000  
504 S159-06         0.6416667 13.500000

In [6]:
# Load necessary libraries
library(dplyr)
library(rlang)

# Define a function to clean the data and remove outliers
remove_outliers <- function(df, column_name) {
  
  # Convert the column name to a symbol
  column_sym <- sym(column_name)
  
  # Clean the data by ensuring the specified column is non-NaN and numeric
  cleaned_data <- df %>%
    filter(!is.na(!!column_sym)) %>%
    mutate(!!column_sym := as.numeric(!!column_sym))
  
  # Calculate Q1 (25th percentile) and Q3 (75th percentile)
  Q1 <- quantile(cleaned_data[[column_name]], 0.15, na.rm = TRUE)
  Q3 <- quantile(cleaned_data[[column_name]], 0.85, na.rm = TRUE)
  
  # Calculate the IQR (Interquartile Range)
  IQR_value <- IQR(cleaned_data[[column_name]], na.rm = TRUE)
  
  # Define lower and upper bounds
  lower_bound <- Q1 - 1.5 * IQR_value
  upper_bound <- Q3 + 1.5 * IQR_value
  
  # Remove rows that are considered outliers
  cleaned_data <- cleaned_data %>%
    filter((.data[[column_name]] >= lower_bound) & (.data[[column_name]] <= upper_bound))
  
  return(cleaned_data)
}

# Example usage:
cleaned_data_act <- remove_outliers(data_act, act_var)
cleaned_data_samples <- remove_outliers(data_samples, Study_variable)
# head(cleaned_samples)
dim(cleaned_data_act)
dim(cleaned_data_samples)


Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Melatonin_pg_g_Stool = as.numeric(Melatonin_pg_g_Stool)`.
Caused by warning:
! NAs introduced by coercion"


[1] 503   3

[1] 583   7

In [7]:
data_act <- cleaned_data_act
data_samples <- cleaned_data_samples


In [8]:
library(dplyr)
#data_samples<-data_samples[,c('Cohort', 'Sample_Code_16s',  'Timepoint', 'Clock_stool',	'Time_since_last_stool',	'Time_since_last_meal',	'Time_since_last_sleep')]
#data_samples<-data_samples[,c('Cohort', 'Sample_Code_16s',  'Timepoint', Study_variable)]


In [9]:
# Function to extract age group based on conditions
 
extract_age_group <- function(sample_id) {

      # Check if the sample ID is NA
  if (is.na(sample_id)) {
    return(NA)
  }

    
  # Check if the sample ID starts with 'S'
  if (startsWith(sample_id, 'S')) {
    # Split the sample ID by '-'
    parts <- strsplit(sample_id, "-")[[1]]
    # Get the second division
    division <- parts[2]
    # Extract the first two characters
    return(as.integer(substr(division, 1, 2)))
  }
    
  # Split the sample ID by '-'
  parts <- strsplit(sample_id, "-")[[1]]
  # Get the second division
  division <- parts[2]
  # Extract the last character
  last_char <- substring(division, nchar(division), nchar(division))
  # Check if the sample ID starts with 'I'
  if (startsWith(sample_id, 'I')) {
    # If so, multiply the last character by 3
    return(as.integer(last_char) * 3)
    #return(0)       # Sleepy cohort ages between 5 and 31 months, so, will be excluded
  } else {
    # Otherwise, return the last character as integer
    return(as.integer(last_char))
  }
}

# Apply the function to create the age group column
data_samples$age_group <- sapply(data_samples$Sample_Code_16s, extract_age_group)

# View the updated data
#print(data)


dim(data_samples)
data_samples[1:3,]

[1] 583   8

Cohort Sample_id Timepoint Day Sample_Code_16s Melatonin_pg_g_Stool
1 SPIN   M001      t2        1   M001-M6         145.53              
2 SPIN   M001      t2        2   M001-06         176.03              
3 SPIN   M002      t2        1   M002-M6         275.78              
  Melatonin_pg_mL age_group
1 NaN             6        
2 NaN             6        
3 NaN             6

In [10]:
# Merge the main data with the power bands data on SubjectID
merged_data <- merge(data_act, data_samples, by = "Sample_Code_16s")
dim(merged_data)

merged_data[1:3,]


[1] 484  10

Sample_Code_16s CFI       Clock_stool Cohort Sample_id Timepoint Day
1 F300-06         0.5783333 6.833333    NUTR   F300      t2        1  
2 F300-06         0.5783333 6.833333    NUTR   F300      t2        2  
3 F300-06         0.5783333 7.250000    NUTR   F300      t2        1  
  Melatonin_pg_g_Stool Melatonin_pg_mL age_group
1  93.59               NaN             6        
2 155.03               NaN             6        
3  93.59               NaN             6

In [11]:
#merged_data_age <- merged_data

if (Age == '3 months'){
    merged_data <- merged_data[merged_data$age_group ==3,]}

if (Age == '6 months'){
    merged_data <- merged_data[merged_data$age_group ==6,]}

if (Age == '12 months'){
    merged_data <- merged_data[merged_data$age_group ==12,]}

dim(merged_data)
merged_data[1:3,]

[1] 484  10

Sample_Code_16s CFI       Clock_stool Cohort Sample_id Timepoint Day
1 F300-06         0.5783333 6.833333    NUTR   F300      t2        1  
2 F300-06         0.5783333 6.833333    NUTR   F300      t2        2  
3 F300-06         0.5783333 7.250000    NUTR   F300      t2        1  
  Melatonin_pg_g_Stool Melatonin_pg_mL age_group
1  93.59               NaN             6        
2 155.03               NaN             6        
3  93.59               NaN             6

In [12]:
# Check for duplicates
if (any(duplicated(merged_data$Sample_Code_16s))) {
  # Remove duplicates, keeping only the first occurrence
  merged_data <- merged_data[!duplicated(merged_data$Sample_Code_16s), ]
  
  # Print a message indicating duplicates were removed
  message("Duplicates were found and removed.")
} else {
  # Print a message indicating no duplicates were found
  message("No duplicates were found.")
}

# View the cleaned dataframe
head(merged_data)
dim(merged_data)

Duplicates were found and removed.



Sample_Code_16s CFI       Clock_stool Cohort Sample_id Timepoint Day
1  F300-06         0.5783333  6.833333   NUTR   F300      t2        1  
5  F302-03         0.6633333 12.000000   NUTR   F302      t1        2  
7  F302-06         0.7083333  9.833333   NUTR   F302      t2        2  
11 F303-03         0.6916667 19.666667   NUTR   F303      t1        2  
15 F304-03         0.5950000 11.250000   NUTR   F304      t1        2  
19 F306-06         0.6833333 18.000000   NUTR   F306      t2        2  
   Melatonin_pg_g_Stool Melatonin_pg_mL age_group
1   93.59               NaN             6        
5  119.22               NaN             3        
7  435.98               NaN             6        
11  30.87               NaN             3        
15  72.51               NaN             3        
19 182.20               NaN             6

[1] 426  10

In [13]:
merged_data$baby_id <- sub("-.*", "", merged_data$Sample_Code_16s)
dim(merged_data)

[1] 426  11

In [14]:
# Load necessary libraries
library(Maaslin2)

if( Age != 'all'){
# Create input_data as a data frame and set row names
input_data1 <- merged_data[, c('CFI'), drop = FALSE]
rownames(input_data1) <- merged_data$Sample_Code_16s

# Create input_metadata as a data frame and set row names
input_metadata1 <- merged_data[, c('Melatonin_pg_g_Stool', 'Clock_stool'), drop = FALSE]
rownames(input_metadata1) <- merged_data$Sample_Code_16s

# Check and clean missing data
input_data1 <- na.omit(input_data1)
input_metadata1 <- na.omit(input_metadata1)

# Ensure output directory exists
dir.create("results", showWarnings = FALSE)

# Run MaAslin2 for multivariate testing
maaslin_results <- Maaslin2(
    input_data = input_data1,
    input_metadata = input_metadata1,
    output = paste0("results/", Age,"/"),
    fixed_effects = c('Melatonin_pg_g_Stool', 'Clock_stool'), # Include independent variables and covariate
    random_effects = NULL, # Use random effects if needed
    normalization = 'NONE', # Or another method if required
    standardize = TRUE
)
    }


Warning message:
"package 'Maaslin2' was built under R version 4.4.2"


In [15]:
# Load necessary library
library(Maaslin2)

if( Age == 'all'){

# Create input_data as a data frame and set row names
input_data1 <- merged_data[, c('CFI'), drop = FALSE]
rownames(input_data1) <- merged_data$Sample_Code_16s

# Create input_metadata as a data frame and set row names
input_metadata1 <- merged_data[, c('Melatonin_pg_g_Stool', 'Clock_stool', 'age_group', 'baby_id'), drop = FALSE]
rownames(input_metadata1) <- merged_data$Sample_Code_16s

# Check and clean missing data
input_data1 <- na.omit(input_data1)
input_metadata1 <- na.omit(input_metadata1)

# Ensure output directory exists
output_dir <- paste0("results/random_effects/", Age, "/", Study_variable, "/")
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Run MaAslin2 for multivariate analysis with age_group as a random effect
maaslin_results <- Maaslin2(
    input_data = input_data1,
    input_metadata = input_metadata1,
    output = output_dir,
    fixed_effects = c('Melatonin_pg_g_Stool', 'Clock_stool'),  # Independent variables
    random_effects = c('baby_id'),  # Specify as a vector
    normalization = 'NONE',  # Change if needed
    standardize = TRUE
)

# Print completion message
cat("MaAslin2 analysis completed. Results saved in:", output_dir, "\n")

    }

[1] "Creating output feature tables folder"
[1] "Creating output fits folder"
[1] "Creating output figures folder"
2025-04-02 15:51:41.821528 INFO::Writing function arguments to log file
2025-04-02 15:51:41.843094 INFO::Verifying options selected are valid
2025-04-02 15:51:41.870473 INFO::Determining format of input files
2025-04-02 15:51:41.871815 INFO::Input format is data samples as rows and metadata samples as rows
2025-04-02 15:51:41.874771 INFO::Formula for random effects: expr ~ (1 | baby_id)
2025-04-02 15:51:41.875873 INFO::Formula for fixed effects: expr ~  Melatonin_pg_g_Stool + Clock_stool
2025-04-02 15:51:41.876864 INFO::Filter data based on min abundance and min prevalence
2025-04-02 15:51:41.87777 INFO::Total samples in data: 424
2025-04-02 15:51:41.878677 INFO::Min samples required with min abundance for a feature not to be filtered: 42.400000
2025-04-02 15:51:41.87995 INFO::Total filtered features: 0
2025-04-02 15:51:41.881019 INFO::Filtered feature names from abundance

boundary (singular) fit: see help('isSingular')



2025-04-02 15:51:43.079366 INFO::Counting total values for each feature
2025-04-02 15:51:43.081231 INFO::Writing filtered data to file results/random_effects/all/Melatonin_pg_g_Stool//features/filtered_data.tsv
2025-04-02 15:51:43.08438 INFO::Writing filtered, normalized data to file results/random_effects/all/Melatonin_pg_g_Stool//features/filtered_data_norm.tsv
2025-04-02 15:51:43.087063 INFO::Writing filtered, normalized, transformed data to file results/random_effects/all/Melatonin_pg_g_Stool//features/filtered_data_norm_transformed.tsv
2025-04-02 15:51:43.090512 INFO::Writing residuals to file results/random_effects/all/Melatonin_pg_g_Stool//fits/residuals.rds
2025-04-02 15:51:43.092183 INFO::Writing fitted values to file results/random_effects/all/Melatonin_pg_g_Stool//fits/fitted.rds
2025-04-02 15:51:43.093776 INFO::Writing extracted random effects to file results/random_effects/all/Melatonin_pg_g_Stool//fits/ranef.rds
2025-04-02 15:51:43.09507 INFO::Writing all results to file 

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


MaAslin2 analysis completed. Results saved in: results/random_effects/all/Melatonin_pg_g_Stool/ 


In [16]:
maaslin_results

$results
  feature             metadata                value        coef      stderr
1     CFI Melatonin_pg_g_Stool Melatonin_pg_g_Stool 0.013067575 0.008260536
2     CFI          Clock_stool          Clock_stool 0.006333535 0.008260536
       pval                 name      qval   N N.not.zero
1 0.1144169 Melatonin_pg_g_Stool 0.2288338 424        424
2 0.4436765          Clock_stool 0.4436765 424        424

$residuals
       F300-06    F302-03   F302-06   F303-03     F304-03    F306-06    F308-03
CFI -0.1373397 0.04985468 0.1066445 0.1107601 -0.09988435 0.07605916 -0.1509635
        F308-06    F309-03    F309-06     F310-03    F310-06   I001-03
CFI -0.01818993 -0.2535645 -0.1037518 -0.08213421 0.07600531 0.2554188
        I002-03  I003-03    I004-03   I005-03    I006-03    I007-03    I008-03
CFI -0.05579356 0.142652 0.08093533 0.0118455 0.02303254 0.07151896 0.05054783
      I010-03    I013-03   I014-03   M001-06    M002-06    M003-03     M004-03
CFI 0.3131062 0.06197871 0.2784619 0.1068623 -0.1246471 -0.1262258 0.003889826
       M004-06     M005-06     M006-03   M006-06     M007-03    M007-06
CFI 0.09810271 0.003040089 -0.03348806 0.0667771 -0.06889925 0.01433602
        M010-03     M010-06    M011-06     M012-06   M013-06     S001-03
CFI -0.07235972 -0.01508393 -0.1772742 -0.09556461 0.2908881 -0.09223315
      S001-12    S003-03    S003-06   S003-12   S004-12     S005-03    S005-06
CFI 0.1770643 -0.2475356 -0.1870152 0.1441422 0.1228568 -0.06279028 -0.1473997
      S005-12    S006-03      S006-06    S006-12    S007-03    S007-06
CFI 0.1727823 -0.1274035 -0.002452199 0.08903488 -0.1040622 -0.2306337
      S007-12   S009-06  S009-12     S010-12    S011-06    S011-12     S012-06
CFI 0.1502525 0.1674774 0.050634 0.001855275 -0.1715035 0.05051065 -0.06626877
       S012-12    S013-03     S013-06  S013-12   S014-03    S014-06    S014-12
CFI -0.1900929 -0.2822481 -0.05751818 0.193616 0.0539411 0.06632802 0.08914835
        S015-03   S015-12     S016-03     S016-06    S016-12   S018-06
CFI -0.08465426 0.1483634 -0.01264524 -0.04506125 0.08990423 0.2241946
      S019-12    S020-03    S020-06     S020-12    S021-03    S021-06
CFI 0.1906912 -0.3392118 0.05351371 -0.05864089 -0.2818193 0.07102277
       S021-12    S022-03     S022-06    S022-12    S023-03   S023-06   S023-12
CFI 0.05709315 -0.1848911 -0.09692275 0.08653858 -0.2693136 0.1236311 0.1801063
       S024-06   S024-12    S025-06   S025-12    S026-03   S026-06  S026-12
CFI 0.02105288 0.2753622 -0.1815005 0.1025311 -0.2650938 0.1056685 0.147527
       S027-06  S027-12    S028-06   S029-12    S030-06   S030-12    S031-03
CFI 0.01480928 0.197944 -0.1111345 0.1542642 0.09296729 0.2628816 -0.1400708
     S031-12    S032-03    S032-06    S033-03   S033-06   S033-12    S034-03
CFI 0.357233 0.07664468 -0.0397028 -0.5178517 0.1171233 0.1232679 -0.0324038
      S034-12    S035-03   S035-06   S035-12    S036-03    S036-06   S037-03
CFI 0.1500661 0.05245414 0.1268441 0.1348112 -0.1694483 -0.2575279 0.1548354
      S037-06   S037-12    S038-03    S038-06    S038-12    S040-03     S040-06
CFI 0.0681285 0.1352238 -0.1249777 -0.1255315 0.07650855 -0.3472238 -0.05244295
      S040-12    S041-03     S041-06    S041-12   S042-06    S043-03
CFI 0.1067896 -0.3514996 -0.08454135 0.03955872 0.1608734 -0.4275538
        S043-06   S043-12    S044-03      S044-06   S044-12     S045-03
CFI -0.02001386 0.1777098 -0.3072515 -0.001936876 0.2479573 -0.02495213
      S045-06   S045-12   S046-03   S046-06    S046-12    S047-03     S047-06
CFI 0.1365951 0.0589069 -0.110292 0.1237786 0.04696819 -0.1756651 -0.08468819
        S047-12    S048-03    S048-06     S048-12    S049-03     S049-06
CFI -0.01993276 -0.1383578 -0.1174369 -0.02232816 -0.3488305 -0.05164965
       S049-12    S050-03   S050-06   S050-12     S051-03    S051-06
CFI 0.09241319 -0.2366081 0.2015579 0.2557781 -0.06711285 -0.1441487
         S051-12    S052-03   S052-12     S053-03    S053-06    S053-12
CFI -0.003638989 0.01943864 0.1898401 -0.07

In [17]:
# Compute Pearson correlation matrix

# Select relevant columns
corr_data <- merged_data[, c("Melatonin_pg_g_Stool", "CFI", "Clock_stool")]

# Compute Pearson correlation matrix
pearson_corr <- cor(corr_data, method = "pearson", use = "complete.obs")

# Define the output file path
output_file <- paste0("results/", Age, "/pearson_correlation.txt")

# Save results to a text file
write.table(pearson_corr, file = output_file, sep = "\t", quote = FALSE, row.names = TRUE, col.names = NA)

# Print correlation matrix
pearson_corr

Warning message in file(file, ifelse(append, "a", "w")):
"cannot open file 'results/all/pearson_correlation.txt': No such file or directory"


ERROR: Error in file(file, ifelse(append, "a", "w")): cannot open the connection
